# Retranscrire ses entretiens avec Whisper

[Whisper](https://github.com/openai/whisper) est le nom donné par Open IA (Chat GPT) à son système de reconnaissance vocale multilingue. Dans ce tutoriel, nous montrant deux façon d'utiliser Whisper:

1. en utilisant Whisper en "local", c'est-à-dire en installant Whisper sur son ordinateur.
2. en utilisant les services Huma-Num


## Whisper en local

Dans ce notebook, nous voyons comment utiliser Whisper -- l'outil de retranscription audio proposé par Open-ai -- en "local" à l'aide d'un script Python. C'est-à-dire qu'on installe whisper et le modèle sur sa machine.

Du fait de la popularité de Whisper, de nombreux tutoriaux qui fleurissent :

- https://www.css.cnrs.fr/whisper-pour-retranscrire-des-entretiens/
- https://analyzingalpha.com/openai-whisper-python-tutorial

###  Quel est l'intérêt ?

1. on maîtrise la "destinée" de ses données : elles restent privées. Il existe toutefois des applications web qui promettent aussi de protéger vos données comme [Whisper Web](https://whisper-web.pascal-mietlicki.fr/).

2. on apprend à charger un modèle. C'est aussi un prétexte pour découvrir la programmation avec Python et Jupyter pour les SHS.

3. Dans une utilisation (très) avancée, cela permet aussi de "fine-tuner" les modèles de Whisper afin qu'ils soient mieux adapter à ses propres corpus.

### les inconvénients

1. Pas toujours simple d'installer Whisper et les librairies nécessaires pour la retranscription (incompatibilité des versions, etc.). Ce n'est pas une solution "clé en main"

2. L'utilisation de Whisper demande beaucoup de ressources. Selon la performance de votre matériel informatique, la retranscription prendra plus ou moins de temps et vous ne pourrez pas nécessairement utiliser les modèles les plus "gros".


Pour éviter au maximum ces problèmes, nous utilisons le SSP Cloud de l'INSEE qui permet d'avoir le même espace de travail. 
Une autre solution, celle qu'on verra ensuite, est de faire appel aux services d'Huma-num lorsqu'on a des fichiers volumineux, de nombreux entretiens ou que l'on souhaite utiliser les modèles les plus performants.

### Installation des librairies Python

L'ensemble des librairies sont déjà installées. Toutefois si vous avez besoin de les réinstaller, il suffit de "décommenter" la ligne de code ci-dessous (i.e. enlever le '#') et de l'éxecuter.

#### Librairie pour manipuler des fichiers audios.

In [ ]:
#!pip install pydub audioop-lts ffmpeg-python

In [53]:
import pydub
from IPython.display import Audio, display #pour écouter les extraits audio avec un notebook jupyter

#### Installation du module Whisper et chargement des modèles

In [54]:
import whisper
import time
import csv


Afin de connaître les modèles disponibles, on peut exécuter la commande suivante :

In [55]:
whisper.available_models()

['tiny.en',
 'tiny',
 'base.en',
 'base',
 'small.en',
 'small',
 'medium.en',
 'medium',
 'large-v1',
 'large-v2',
 'large-v3',
 'large',
 'large-v3-turbo',
 'turbo']

Les modèles terminant par ".en" sont ceux qui ont été entrainés uniquement sur des corpus anglophone. Les autres ont été entrainés sur des coprus multilingues.
Dans cet atelier, nous allons testés les quatre modèle suivants : "tiny", "medium", "large" et "large-v3".

In [56]:
tiny_model = whisper.load_model("tiny")
medium_model = whisper.load_model("medium")
large_model = whisper.load_model("large")
largev3_model = whisper.load_model("large-v3")

On peut déjà constater que la taille des modèles augmentent ainsi que le temps de chargement

### Charger et segmenter un mp3

On commence par charger l'entretien (au format mp3). On créer pour cela un dossier "records/" dans lequel on télécharge notre fichier mp3.

In [57]:
itw = pydub.AudioSegment.from_mp3("../records/grazia_borrini_07-06-18_part-1.MP3") 

La ligne ci-dessous permet d'extraire un morceau d'entretien d'une minute entre la 10e et la 11e minute. La briéveté de l'extrait permettra de faire des tests avec plusieurs modèles.

In [58]:
extract = itw[600*1000:660*1000]

On enregistre notre extrait en exécutant la ligne ci-dessous. On remarque qu'un nouveau fichier sample.mp3 est apparu dans le dossier "records/".
![](../figures/Capture_ecran_2025-12-08_15-33-24_sample.png)

In [59]:
pydub.AudioSegment.export(extract,"../records/sample.mp3") #

<_io.BufferedRandom name='../records/sample.mp3'>

### Pour écouter l'extrait audio

In [60]:
display(Audio('../records/sample.mp3', autoplay=True))

## Retranscrire l'extrait

Nous allons maintenant tester la retranscription sur notre extrait en utilisant différents modèles. L'objectif est de comparer les résultats et définir le "meilleur" modèle en foction de la qualité de la transcription et du temps.
Le résultat est retourné sous la forme d'une structure json.

À noter qu'on utilise deux boucles `for` dans la cellule ci-après. La première permet de répéter l'opération de retranscription pour chaque modèle testé. La seconde d'afficher le résultat de la retranscription.

Les boucles `for` en Python sont essentielles pour itérer sur des séquences et effectuer des opérations répétitives de manière efficace.

In [61]:

for n, model in enumerate([tiny_model, medium_model, large_model, largev3_model]):
    start = time.time() # on lance le chronomètre
    retranscribe = model.transcribe("../records/sample.mp3", verbose=None, fp16=False)
    print(f"Temps écoulé : ", time.ctime(time.time() - start)[11:19])
    if n == 1:
        result_medium = retranscribe
    for n, x in enumerate(retranscribe["segments"]):
        print(f"Segment {n} : {x["text"]}\n")

Temps écoulé :  00:00:06
Segment 0 :  de générer ça. Parfois, je pense que je vais des opinions à rétrailer, à rétrailer,

Segment 1 :  je ne sais pas. Car je crois que ça dépend beaucoup de tout viendre, de l'éducation,

Segment 2 :  de ses que tu as étudié. Je sais que pour moi ça a été incroyable, d'elle lire,

Segment 3 :  un peu de philosophie et à la holissée, mais surtout de lire une it,

Segment 4 :  je me rappelle vraiment très bien, d'avoir les histoires de la possibilité des créés

Segment 5 :  et propos, et voilà. Ça m'a bouleversé énormément, ça m'a ouvert la tête comme une pomme,

Segment 6 :  et je me sais dit, ou wow. Alors là, c'est vraiment possible d'analyser et

Segment 7 :  des critiqués et etc.

Temps écoulé :  00:00:59
Segment 0 :  de générer ça. Parfois je pense que j'ai des opinions un peu arriérées, car je crois

Segment 1 :  que ça dépend beaucoup d'où tout vient, de l'éducation, de ce que tu as étudié.

Segment 2 :  Je sais que pour moi ça a été incroyable d

Dans la cellule ci-dessus, on utilise une boucle `for` pour afficher le résultat de la retranscription pour chacun des modèles.



On constate que les résultats produits par le modèle "Tiny" sont très mauvais. Ils sont nettement meilleurs avec le modèle médium.

On constate également que la différence de résultats entre les modèles médium et large n'est pas flagrante. Si on prend en compte la durée de la retranscription, on peut s'interroger sur l'intérêt d'utiliser le modèle large. 

Enfin, dans tous les cas, les modèles produisent des erreurs. Le plus souvent, ce sont des erreurs de transcription : le texte ne correspond pas à ce qui est dit. Il arrive aussi que Whisper omette de retranscrire un passage. L'utilisation de Whisper ne dispense donc pas d'écouter les entretiens. Ainsi on va faciliter le travail de retranscription sans totalement perdre le lien avec le matériaux brut.

## Mise en forme et sauvegarde

(inspiration Yacine Chitour)

La sortie obtenue après la retranscription prend la forme d'un dictionnaire. On va donc "mettre en forme" cette sortie pour qu'on puisse sauvegarder le résultat dans des formats plus facile à manipuler avec des logiciels de traitements de texte.


In [62]:
import pandas as pd
speech = pd.DataFrame.from_dict(result_medium['segments'])
speech.head()

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob
0,0,0,0.00,13.64,de générer ça. Parfois je pense que j'ai des ...,"[50364, 368, 14575, 21824, 2788, 13, 3457, 69,...",0.0,-0.196777,1.305970,0.286383
1,1,0,13.64,21.72,"que ça dépend beaucoup d'où tout vient, de l'...","[51046, 631, 2788, 45768, 8796, 274, 6, 78, 50...",0.0,-0.196777,1.305970,0.286383
2,2,2172,21.72,29.56,Je sais que pour moi ça a été incroyable de l...,"[50364, 2588, 11757, 631, 2016, 7748, 2788, 25...",0.0,-0.199966,1.388298,0.363143
3,3,2172,29.56,38.12,mais surtout de lire Nietzsche. Je me rappell...,"[50756, 2420, 19903, 368, 43254, 36583, 89, 12...",0.0,-0.199966,1.388298,0.363143
4,4,2172,38.12,47.24,de la possibilité de créer tes propres valeur...,"[51184, 368, 635, 24145, 5066, 368, 32062, 200...",0.0,-0.199966,1.388298,0.363143


In [63]:
def convertir(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
 
    return f"{h:02d}:{m:02d}:{s:02d}"

In [64]:
convertir(100000)

'27:46:40'

In [29]:
time.ctime(13.64)

'Thu Jan  1 00:00:13 1970'

In [65]:
"txt" in ["txt", "csv"]

True

In [69]:
def save_transcript(transcription, file_name="", extention=[]):
    print(len(extention))
    if len(extention) == 0:
        guess_from_name_file = file_name.split(".")[-1]
        print(guess_from_name_file)
        if guess_from_name_file in ["txt", "csv", "srt"]:
                extention.append(guess_from_name_file)
        else:
            print("Veuillez préciser l'extension de sotie en choisissant parmi les trois formats suivants : '.txt', '.csv', '.srt'!\n")
            
            
    else:
        pass

    for n, x in enumerate(extention):
        file_name_without_extention = file_name.split(".")[0]
        if x == "txt":
            file_name_with_ext = f"{file_name_without_extention}.{x}"
            with open(file_name_with_ext, 'w', encoding='utf-8') as f:
                for segment in transcription["segments"]:
                    start_time = convertir(segment['start'])
                    end_time = convertir(segment['end'])
                    f.write(f"{start_time} - {end_time}: {segment['text']}\n")
        elif x == "srt":
            file_name_with_ext = f"{file_name_without_extention}.{x}"
            with open(file_name_with_ext, 'w', encoding='utf-8') as f:
                for segment in transcription["segments"]:
                    start_time = convertir(segment['start'])
                    end_time = convertir(segment['end'])
                    f.write(f"{start_time} --> {end_time}\n{segment['text']}\n\n")
        elif x == "csv":
            file_name_with_ext = f"{file_name_without_extention}.{x}"
            with open(file_name_with_ext , 'w', newline='') as csvfile:
                fieldnames = ['start', 'end','text']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for segment in transcription["segments"]:
                    start_time = convertir(segment['start'])
                    end_time = convertir(segment['end'])
                    writer.writerow({'start': start_time, 'end': end_time, 'text':segment['text']})

In [24]:
# Chemin où vous souhaitez enregistrer votre transcription d'entretien en format .txt
entretien_transcrit = "entretien.txt"
 
# Enregistrement de la transcription
with open(entretien_transcrit , 'w', encoding='utf-8') as f:
    for segment in result_medium["segments"]:
        start_time = convertir(segment['start'])
        end_time = convertir(segment['end'])
        f.write(f"{start_time} - {end_time}: {segment['text']}\n")

In [71]:
file_name = "2026-01-09_itw_borrini1.tsv"

save_transcript(result_medium, file_name= file_name, extention=["csv", "txt", "srt"])

3


In [25]:
print(open("entretien.txt","r").read())

00:00:00 - 00:00:13:  de générer ça. Parfois je pense que j'ai des opinions un peu arriérées, car je crois
00:00:13 - 00:00:21:  que ça dépend beaucoup d'où tout vient, de l'éducation, de ce que tu as étudié.
00:00:21 - 00:00:29:  Je sais que pour moi ça a été incroyable de lire un peu de philosophie au lycée,
00:00:29 - 00:00:38:  mais surtout de lire Nietzsche. Je me rappelle vraiment très bien d'avoir lu les histoires
00:00:38 - 00:00:47:  de la possibilité de créer tes propres valeurs. Ça m'a bouleversée énormément,
00:00:47 - 00:00:54:  ça m'a ouvert la tête comme une pomme et je me suis dit « waouh ! » À l'heure là,
00:00:54 - 00:00:59:  c'est vraiment possible d'analyser, de critiquer, etc.



Tester l'erreur 

- https://github.com/jitsi/jiwer